In [1]:
struct Mystruct
  num::Integer
  str::String
end

In [2]:
m = Mystruct(13, "hello")

Mystruct(13, "hello")

In [3]:
m.num

13

In [4]:
m.str

"hello"

In [5]:
fieldnames(Mystruct)

(:num, :str)

In [6]:
dump(m)

Mystruct
  num: Int64 13
  str: String "hello"


In [7]:
m.str = "goodbye"

ErrorException: setfield!: immutable struct of type Mystruct cannot be changed

In [8]:
mutable struct MutableStruct
  a::Float64
  b::Integer
end

In [9]:
s = MutableStruct(1,2)

MutableStruct(1.0, 2)

In [10]:
s.a=4.5

4.5

In [11]:
s.b = 7.1

InexactError: InexactError: Int64(7.1)

In [3]:
ranks = ['A','2','3','4','5','6','7','8','9','T','J','Q','K']
suits = ['\u2660','\u2661','\u2662','\u2663']

4-element Vector{Char}:
 '♠': Unicode U+2660 (category So: Symbol, other)
 '♡': Unicode U+2661 (category So: Symbol, other)
 '♢': Unicode U+2662 (category So: Symbol, other)
 '♣': Unicode U+2663 (category So: Symbol, other)

In [13]:
struct Card
  rank::Integer
  suit::Integer
end

In [14]:
c = Card(3,2)

Card(3, 2)

In [15]:
c.rank

3

In [16]:
c.suit

2

In [6]:
Base.show(io::IO, c::Card) = print(io, "$(ranks[c.rank])$(suits[c.suit])")

In [18]:
c

3♡

In [6]:
Card(-10,78)

ArgumentError: ArgumentError: The rank must be an integer between 1 and 13.

In [4]:
struct Card
  rank::Int
  suit::Int

  # construct a card based on the rank and suit
  function Card(r::Int,s::Int)
    1 <= r <=13  || throw(ArgumentError("The rank must be an integer between 1 and 13."))
    1 <= s <= 4  || throw(ArgumentError("The suit must be an integer between 1 and 4."))
    new(r,s)
  end

  # construct a card based on the number in a deck
  function Card(i::Int)
    1 <= i <= 52 || throw(ArgumentError("The argument must be an integer between 1 and 52"))
    # mod(i,13)==0 ? new(13,div(i,13)) : new(mod(i,13),div(i,13)+1)
    new(mod1(i,13), div(i-1,13)+1)
  end
end

In [8]:
Card(-10,12)

ArgumentError: ArgumentError: The rank must be an integer between 1 and 13.

In [7]:
map(i -> Card(mod1(i,13), div(i-1,13)+1) , 1:52)

52-element Vector{Card}:
 A♠
 2♠
 3♠
 4♠
 5♠
 6♠
 7♠
 8♠
 9♠
 T♠
 ⋮
 5♣
 6♣
 7♣
 8♣
 9♣
 T♣
 J♣
 Q♣
 K♣

In [8]:
Card(3,2)

3♡

In [9]:
Card(42)

3♣

In [10]:
struct Hand
  cards::Vector{Card}
end

In [11]:
Base.show(io::IO,h::Hand) = print(io, "[$(join(h.cards,", "))]")

In [12]:
h1=Hand([Card(2,3),Card(12,1),Card(10,1),Card(10,4),Card(5,2)])

[2♢, Q♠, T♠, T♣, 5♡]

In [19]:
struct Polynomial{T <: Number}
  coeffs::Vector{T}
end

In [24]:
poly1=Polynomial([1,2,3])
poly2=Polynomial([1.0,2.0,3.0])
poly3=Polynomial([2//3,3//4,5//8])
poly4=Polynomial([im,2+0im,3-2im,-im])
poly5=Polynomial([n for n=1:6])

1 x^0 + 2 x^1  + 3 x^2  + 4 x^3  + 5 x^4  + 6 x^5 

In [25]:
poly3

2//3 x^0 + 3//4 x^1  + 5//8 x^2 

In [23]:
function Base.show(io::IO, p::Polynomial)
  print(io, mapreduce(i -> "$(p.coeffs[i]) x^$(i-1)", (str, term) -> "$str + $term " , 1:length(p.coeffs)))
end

In [19]:
poly5

  1 x^0 + 2 x^1 + 3 x^2 + 4 x^3 + 5 x^4 + 6 x^5

In [20]:
poly3

  2//3 x^0 + 3//4 x^1 + 5//8 x^2

In [26]:
poly1+poly2

MethodError: MethodError: no method matching +(::Polynomial{Int64}, ::Polynomial{Float64})
The function `+` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  +(!Matched::Rational{BigInt}, !Matched::Rational{BigInt})
   @ Base gmp.jl:1056
  +(!Matched::Bool, !Matched::Complex{Bool})
   @ Base complex.jl:308
  ...


In [27]:
import Base.+

In [28]:
function +(p1::Polynomial{<:Number},p2::Polynomial{<:Number})
  Polynomial(p1.coeffs+p2.coeffs)
end

+ (generic function with 199 methods)

In [29]:
poly1+poly2

2.0 x^0 + 4.0 x^1  + 6.0 x^2 

In [30]:
function eval(poly::Polynomial{<:Number},x::Number)
  reduce((val,i) -> val + poly.coeffs[i]*x^(i-1), 1:length(poly.coeffs))
end

eval (generic function with 2 methods)

In [31]:
eval(poly1,-2)

9

In [32]:
function eval(poly::Polynomial{<:Number}, x::Number)
  reduce((val,c) -> x*val+c, reverse(poly.coeffs))
end

eval (generic function with 2 methods)

In [33]:
eval(poly1,-2)

9

In [37]:
struct Root
  root::Float64    #  approximate value of the root
  x_eps::Float64   #  estimate of the error in the x variable
  f_eps::Float64   #  function value at the root f(root)
  num_steps::Int   #  number of steps the method used
  converged::Bool  #  whether or not the stopping criterion was reached
  max_steps::Int   #  the maximum number of steps allowed
end

In [36]:
using ForwardDiff

In [69]:
function newton(f::Function, x0::Number; tol::Real = 1e-6, max_steps = 10)
  local dx
  for i = 1:max_steps
    dx = f(x0)/ForwardDiff.derivative(f, x0)
    x0 -= dx
    abs(dx) < 1e-6 && return Root(x0, dx, f(x0), i, true, max_steps)
  end
  Root(x0, dx, f(x0), max_steps, false, max_steps)
end

newton (generic function with 1 method)

In [70]:
newton(x->x^2-2, 1)

The root is approximately x̂ = 1.4142135623730951
An estimate for the error is 1.5947429102833119e-12
with f(x̂) = 4.440892098500626e-16
which took 5 steps

In [71]:
function Base.show(io::IO,r::Root)
  str = r.converged ? """The root is approximately x̂ = $(r.root)
    An estimate for the error is $(r.x_eps)
    with f(x̂) = $(r.f_eps)
    which took $(r.num_steps) steps""" :
    """The root was not found within $(r.max_steps) steps.
    Currently, the root is approximately x̂ = $(r.root).
    An estimate for the error is $(r.x_eps)
    with f(x̂) = $(r.f_eps)."""
  print(io,str)
end

In [72]:
r = newton(x->x^2-2, 1)

The root is approximately x̂ = 1.4142135623730951
An estimate for the error is 1.5947429102833119e-12
with f(x̂) = 4.440892098500626e-16
which took 5 steps

In [73]:
newton(x->x^2+1,1.1)

The root was not found within 10 steps.
Currently, the root is approximately x̂ = 0.030421004027873844.
An estimate for the error is 1.0004626117382218
with f(x̂) = 1.0009254374860639.